In [1]:
)load isolate
isolate.Config 'listen' 1

/opt/mdyalog/19.0/64/unicode/ws/isolate.dws ⍝ saved Wed Sep 4 22:43:51 2024
 
 For information on how to use Futures and Isolates, 
 see the manual "Parallel Language Features", 
 which can be found in the Dyalog Documentation Centre 
 at http://docs.dyalog.com

0

In [2]:
SNRs ← (¯35 + (⍳50)÷2)
iso ← ø¨(≢SNRs)⍴⊂''
iso.SNR ← SNRs
SNR ← 0

In [3]:
'InitCauseway' 'View' ⎕CY'sharpplot'

In [4]:
]dinput
plot ← {
    do  ← {⍎'⍺⍺ ⍵ ⋄ ⍵' ⋄ ⍺⍺}
    _   ← InitCauseway do ⍬
    sp  ← ⎕NEW Causeway.SharpPlot
    ⍝sp.YAxisStyle ← Causeway.YAxisStyles.LogScale + Causeway.YAxisStyles.ClampToAxis
    sp.LineGraphStyle ← Causeway.LineGraphStyles.GridLines
    _   ← sp.DrawLineGraph do ⍵
    sp.SavePdf ⍺ Causeway.PdfMode.ShowOutlines
    ⍝svg ← sp.RenderSvg Causeway.SvgMode.FixedAspect
    ⍝_   ← 'Reaction Times' (3500⌶) svg
}

In [5]:
]dinput
fft ← {
    k    ← 2÷⍨≢⍵
    1>k : ⍵
    0≠1 | 2⍟≢⍵: 'la taille doit etre un mult de 2'
    odd  ← ∇((≢⍵)⍴1 0)/⍵
    even ← ∇((≢⍵)⍴0 1)/⍵
    exp  ← *(0J¯2×(○1)×(¯1+⍳k)÷≢⍵)
    T    ← even × exp
    (odd+T), (odd-T)
}

In [4]:
iso.SF ← SF ← 8                 ⍝ spread factor
iso.ns ← ns ← 2*12              ⍝ nombre de samples at least 19
iso.B  ← B  ← 125e3             ⍝ bandwitdh
iso.f0 ← f0 ← 0
iso.T  ← T  ← 1 ÷ B             ⍝ periode
iso.Ts ← Ts ← T × 2*SF          ⍝ periode de symbole
iso.t  ← ⊂(¯1+⍳ns)×Ts÷ns   ⍝ variable du temps
t  ← (¯1+⍳ns)×Ts÷ns
iso.Ps ← Ps ← 1

In [5]:
Re  ← { (⍵ + +⍵) ÷ 2 }

In [6]:
]dinput
chirp ← {
    s   ← ⍵
    f   ← ((2*SF) | (t÷T) + s) × (B÷(2*SF))
    phi ← ○2 × +\ (f0+f) × Ts÷ns
    *(0J1 × phi)
}

In [7]:
]dinput
chirp_string ← {
    w ← ⎕UCS ⍵
    ∊chirp¨w
}

In [7]:
]load ../pynapl/pynapl/Py
py ← ⎕NEW Py.Py
plt ← py.Import 'matplotlib.pyplot'

#.Py

In [33]:
]dinput
gauss ← {
   depth ← 2*32                             ⍝ profondeure de aletoirement
   (x y) ← ⊂[1+⍳⍴,⍵](?(2,⍵)⍴ depth) ÷ depth ⍝ deux variables aleatoires dans ]0;1]
   ((¯2×⍟x) * 0.5) × *(0J1 × ○2 × y)        ⍝ methode de Box-Muller
}

In [10]:
noise ← { ((Ps × 10*SNR÷¯10) * 0.5) × gauss ≢⍵ }

In [11]:
chop ← { s ← ≢⍵ ⋄ ws ← s÷2 ⋄ sk ← 8 ⋄ ( (1+s-ws) ⍴ ((sk⍴0), 1))/ (ws ,/ ⍵) }

In [12]:
spectrogram ← { N ← 1 ⋄ {((≢⍵)÷N)↑⍵} |fft ¨ chop ⍵ (,⍣N)⍬ }

In [ ]:
sym ← 25
c   ← chirp sym
nl ← ⎕UCS 10
{(Re c[⍵]) ',' (t[⍵]) (nl)}¨(⍳ns)

In [74]:
sym ← 420
c   ← chirp sym
r   ← c + noise c

⍝plot r+c t
spe ← spectrogram r

In [81]:
plt.imshow ⊂↓⌽⍉↑spe
plt.show ⍬

In [13]:
dechirp ← { s ← {⍵ ≡ ⍬: 0 ⋄ ⍵}⍵ ⋄ ⊖+chirp s }

In [41]:

plt.imshow ⊂ spectrogram (c×dechirp ⍬)
plt.show ⍬

In [75]:
plot |fft r×dechirp ⍬

Created by Causeway SVG engine - SharpPlot v3.71.0 
 
 Paint the paper ===== 
 
 
 
 Border ===== 
 Region ===== 
 X-Axis Ticks ===== 
 X-Axis tickmarks 
 
 Y-Axis Ticks ===== 
 Y-Axis tickmarks 
 
 Axes ===== 
 
 Y-axis labels 
 
 0 
 10 
 20 
 30 
 40 
 50 
 60 
 70 
 80 
 90 
 100 
 110 
 120 
 130 
 140 
 
 for X-axis labels 
 
 0 
 10 
 20 
 30 
 40 
 50 
 60 
 70 
 80 
 90 
 100 
 110 
 120 
 130 
 
 Heading, subheading and footnotes ===== 
 Start of Line Chart =========== 
 Points follow ... 
 Line 
 
 <path d="M4043 2061L4015 2139 3986 2226 3958 1486 3929 2350 3901 1966 3872 2108 3844 1854 3815 1489 3787 1494 3758 2062 3730 2619 3702 2345 3673 2525 3645 1972 3616 2244 3588 1780 3559 2090 3531 2108 3502 537 3474 2185 3445 1797 3417 1906 3388 1628 3360 2486 3332 1612 3303 1881 3275 2488 3246 1862 3218 1703 3189 2133 3161 1948 3132 2431 3104 2116 3075 2338 3047 2509 3018 1945 2990 1973 2962 1754 2933 2576 2905 1837 2876 2290 2848 1768 2819 2011 2791 1757 2762 2291 2734 2323 2705 1544 2677 1932 2648 2518 2620 1993 2592 1563 2563 1726 2535 2391 2506 2369 2478 1979 2449 2549 2421 1478 2392 1843 2364 2228 2335 1726 2307 2378 2278 2550 2250 1384 2222 2419 2193 1963 2165 1748 2136 1439 2108 1996 2079 2049 2051 1805 2022 2221 1994 1776 1965 2224 1937 2217 1908 2089 1880 2277 1852 1606 1823 1874 1795 1868 1766 2411 1738 2116 1709 1867 1681 1910 1652 2298 1624 1687 1595 2145 1567 2534 1538 2210 1510 2012 1482 2403 1453 711 1425 2309 1396 2242 1368 960 1339 2313 1311 1206 1282 2088 1254 1352 1225 1303 1197 1980 1168 2409 1140 1766 1112 1593 1083 2499 1055 1946 1026 1920 998 1236 969 1492 941 1773 912 2039 884 1798 855 1353 827 2054 798 2159 770 2279 742 1538 713 894 685 1802 656 2360 628 1951 599 2513 571 1671 542 2282 514 1544 485 2209 457 1790 428 1838" />
 
 Reset to original origin

In [14]:
demodulate ← { ¯1 + ⊃⍒|fft ⍵×dechirp ⍬ }

In [15]:
]dinput
check ← {
    c   ← chirp ⍵
    r   ← c + noise c
    ⍵ = demodulate r
}

In [16]:
_←iso.⎕FX ⊂⎕CR 'fft'
_←iso.⎕FX ⊂⎕CR 'check'
_←iso.⎕FX ⊂⎕CR 'chirp'
_←iso.⎕FX ⊂⎕CR 'noise'
_←iso.⎕FX ⊂⎕CR 'demodulate'
_←iso.⎕FX ⊂⎕CR 'gauss'
_←iso.⎕FX ⊂⎕CR 'dechirp'

In [17]:
simulate ← { r ← ⍵ ÷⍨ ⍵ - +/check¨?⍵⍴¯1+2*SF ⋄ r = 0 : 100÷⍵ ⋄ r×100 }

In [18]:
_←iso.⎕FX ⊂⎕CR 'simulate'

In [19]:
varieSF ← { SF ⊢← ⍵ ⋄ ns ⊢← 2*SF ⋄ Ts ⊢← T × 2*SF ⋄ t ⊢← (¯1+⍳ns)×Ts÷ns ⋄ simulate 1000 }

In [20]:
_←iso.⎕FX ⊂⎕CR 'varieSF'

In [21]:
SFhh  ← iso.(varieSF¨7+⍳9)

In [23]:
plot (↓⍉↑SFhh) SNRs

Created by Causeway SVG engine - SharpPlot v3.71.0 
 
 Paint the paper ===== 
 
 
 
 Border ===== 
 Region ===== 
 X-Axis Ticks ===== 
 X-Axis Grid 
 
 X-Axis tickmarks 
 
 Y-Axis Ticks ===== 
 Y grid 
 
 Y-Axis tickmarks 
 
 Axes ===== 
 
 Y-axis labels 
 
 0.1 
 1 
 10 
 100 
 
 for X-axis labels 
 
 -36 
 -34 
 -32 
 -30 
 -28 
 -26 
 -24 
 -22 
 -20 
 -18 
 -16 
 -14 
 -12 
 -10 
 
 Heading, subheading and footnotes ===== 
 Start of Line Chart =========== 
 Points follow ... 
 Line 
 
 
 
 Points follow ... 
 Line 
 
 
 
 Points follow ... 
 Line 
 
 
 
 Points follow ... 
 Line 
 
 
 
 Points follow ... 
 Line 
 
 
 
 Points follow ... 
 Line 
 
 
 
 Points follow ... 
 Line 
 
 
 
 Points follow ... 
 Line 
 
 
 
 Points follow ... 
 Line 
 
 
 
 Reset to original origin

Could not load bridge interface functions. (Error 0x00000000: "")

In [70]:
cfar ← { N ← 10 ⋄ cut ← ((-N)↓N↓⍵) ⋄ cut > (N÷2) ÷⍨ cut -⍨ +/¨(N+1+N) ,/ ⍵ }